In [ ]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez
from Bio import SeqIO

In [ ]:
Entrez.email = 'greg.clinton@gmail.com'

In [ ]:
# http://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html

# How many genes does e. coli have? What are their names?
# How many species under homo?
# sra vs nucleotide, gene, ref_seq

In [3]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
db = 'nuccore'

In [ ]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
record = Entrez.read(Entrez.esearch(db = dv, term = q))
ids = ','.join(record['IdList'][:5])
handle = Entrez.efetch(db = 'nuccore', id = ids, rettype = 'gb', retmode = 'text')
records = SeqIO.parse(handle, 'gb')

In [ ]:
for record in records:
    print record.name

In [28]:
import requests

email = 'greg.clinton@gmail.com'

def entrez(path):
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    url += path
    url += '&tool=entrez&email=' + email
    return requests.get(url).text

def esearch(db, term):
    return entrez('esearch.fcgi?db=%s&term=%s' % (db, term))

def einfo(db = None):
    return entrez('einfo.fcgi?' + ('x=0' if db == None else 'version=2.0&db=%s' % db))

einfo('nuccore')

u'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20130322//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20130322/einfo.dtd">\n<eInfoResult>\n\t<DbInfo>\n\t<DbName>nuccore</DbName>\n\t<MenuName>Nucleotide</MenuName>\n\t<Description>Core Nucleotide db</Description>\n\t<DbBuild>Build160828-0120m.1</DbBuild>\n\t<Count>209863088</Count>\n\t<LastUpdate>2016/08/29 09:28</LastUpdate>\n\t<FieldList>\n\t\t<Field>\n\t\t\t<Name>ALL</Name>\n\t\t\t<FullName>All Fields</FullName>\n\t\t\t<Description>All terms from all searchable fields</Description>\n\t\t\t<TermCount>2159331205</TermCount>\n\t\t\t<IsDate>N</IsDate>\n\t\t\t<IsNumerical>N</IsNumerical>\n\t\t\t<SingleToken>N</SingleToken>\n\t\t\t<Hierarchy>N</Hierarchy>\n\t\t\t<IsHidden>N</IsHidden>\n\t\t\t<IsTruncatable>Y</IsTruncatable>\n\t\t\t<IsRangable>N</IsRangable>\n\t\t</Field>\n\t\t<Field>\n\t\t\t<Name>UID</Name>\n\t\t\t<FullName>UID</FullName>\n\t\t\t<Description>Unique number assigned to each seque

In [ ]:
def einfo
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi